# About: 運用環境の移行--コンテナのパッケージアップデート

---

検証環境の更新内容を利用して運用環境の移行を行います。

## 概要

検証した設定を用いて新しい運用環境を構築し、運用環境の切り替えを行います。

![運用環境の移行](images/moodle-122-01.png)

### グループ名の指定

このNotebookの操作対象となる UnitGroup名を指定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `UnitGroup`名に対応する group_varsファイルが存在していることを確認します。エラーになる場合は、指定したUnitGroup名が正しくないと考えられます。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

UnitGroupに属する VCノードに対して Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping
!ansible {ugroup_name} -b -a 'whoami'

検証環境に関するパラメータが group_vars に保存されていることを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

if 'update_container_target' not in gvars:
    raise RuntimeError("ERROR")
if 'update_project_tag' not in gvars:
    raise RuntimeError("ERROR")

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

#### チェック

入力されたアクセストークンが正しいことを、VCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

### 準備

これまでに他のNotebookで設定したパラメータを読み込む処理などを行います。

group_varsファイルに保存されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

## 新しい運用環境の作成

検証環境を元にした新しい運用環境を作成します。

![新しい運用環境の作成](images/moodle-122-02.png)

### メンテナンスモードへの切り替え

新しい運用環境を作成するまでの間に、現在の運用環境に新たなデータが書き込まれないようにするために、メンテナンスモードに切り替えます。

In [ ]:
moodle_container = f'moodle-{gvars["project_tag"]}'
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose exec -T {moodle_container} \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

### コンテナのコミット

検証環境のMoodleコンテナから、新しい運用環境で使用するコンテナイメージ作成します。

In [ ]:
source_container = f'{gvars["update_container_target"]}test-{gvars["update_project_tag"]}'
target_container_image = f'local/{gvars["update_container_target"]}:{gvars["update_project_tag"]}'
print(f'docker commit {source_container} {target_container_image}')

!ansible {ugroup_name} -a \
    'docker commit {source_container} {target_container_image}'

コンテナイメージが作成されたことを確認します。

In [ ]:
!ansible {ugroup_name} -a \
    'docker images {target_container_image}'

### 検証環境コンテナの停止

検証環境のコンテナを停止します。

現在のコンテナの実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

検証環境のコンテナを停止します。

In [ ]:
test_containers = [f'{x}test-{gvars["update_project_tag"]}' for x in ['moodle', 'db']]
print(test_containers)

!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose rm -f -s {" ".join(test_containers)}'

停止後のコンテナの実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

### 論理ボリュームの複製

新しい運用環境のための論理ボリュームを作成します。

まず、現在の運用環境から、データベース用論理ボリューム、moodledata用論理ボリュームのスナップショットを作成します。

In [ ]:
project_tag = gvars['project_tag']
update_project_tag = gvars['update_project_tag']

# スナップショットを作成する
for name in ['db', 'moodle']:
    vg = name
    lv = f'{project_tag}_{name}'
    snapshot = f'{update_project_tag}_{name}'
    print(f'vg={vg} lv={lv} snapshot={snapshot}')
    !ansible {ugroup_name} -b -m lvol -a \
        'vg={vg} lv={lv} snapshot={snapshot} opts="-kn"'

検証環境のPHP用論理ボリュームのスナップショットを作成します。

In [ ]:
vg = 'moodle'
lv = f'{update_project_tag}_php_test'
snapshot = f'{update_project_tag}_php'
print(f'vg={vg} lv={lv} snapshot={snapshot}')
!ansible {ugroup_name} -b -m lvol -a \
    'vg={vg} lv={lv} snapshot={snapshot} opts="-kn"'

スナップショットが作成されたことを確認するために、論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

作成したスナップショットをマウント可能にするために、論理ボリュームのアクティベートとUUIDの再作成を行います。

In [ ]:
# LVとVGとの対応
lv_map = {
    'moodle': 'moodle',
    'php': 'moodle',
    'db': 'db',
}

for name, vg in lv_map.items():
    snapshot = f'{update_project_tag}_{name}'
    dev = f'/dev/mapper/{vg}-{snapshot}'
    print(f'snapshot={snapshot} dev={dev}')
    !ansible {ugroup_name} -b -m shell -a \
        'lvchange -ay {vg}/{snapshot} && \
         mount -o nouuid {dev} /mnt && \
         umount /mnt && \
         xfs_admin -U generate {dev}'

スナップショットがアクティベートされたことを確認するために、論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

作成したボリュームをマウントします。

In [ ]:
mount_point = {
    'moodle': f'/opt/moodle/moodle-{update_project_tag}/data/moodledata',
    'php': f'/opt/moodle/moodle-{update_project_tag}/data/php',
    'db': f'/opt/moodle/db-{update_project_tag}/data',
}

for name, vg in lv_map.items():
    snapshot = f'{update_project_tag}_{name}'
    dev = f'/dev/mapper/{vg}-{snapshot}'
    print(f'path={mount_point[name]} src={dev}')
    !ansible {ugroup_name} -b -m mount -a \
        'path={mount_point[name]} src={dev} fstype=xfs state=mounted'

### 設定ファイルの複製

検証環境の設定ファイルをコピーして新しい運用環境の設定ファイルを作成します。

In [ ]:
for name in ['moodle', 'db']:
    src = f'{name}-{project_tag}/conf'
    dst = f'{name}-{update_project_tag}'
    print(f'src={src} dst={dst}')
    !ansible {ugroup_name} -b -a \
        'chdir=/opt/moodle cp -a {src} {dst}'

## 検証環境の削除

新しい運用環境が作成され、不要となった検証環境を削除します。

![検証環境の削除](images/moodle-122-03.png)

### 検証環境用論理ボリュームの削除

検証環境の論理ボリュームを削除します。

現在の論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

削除対象となる論理ボリュームを確認します。

In [ ]:
remove_volumes = dict([
    (f'{update_project_tag}_{name}_test', vg)
    for name, vg in lv_map.items()
])

for x in sorted(remove_volumes.keys()):
    print(x)

削除対象となる論理ボリュームを umount します。

In [ ]:
for lv, vg in remove_volumes.items():
    dev = f'/dev/mapper/{vg}-{lv}'
    print(f'umount {dev}')
    !ansible {ugroup_name} -b -a \
        'umount {dev}'

論理ボリュームを削除します。

In [ ]:
for lv, vg in remove_volumes.items():
    print(f'vg={vg} lv={lv}')
    !ansible {ugroup_name} -b -m lvol -a \
        'vg={vg} lv={lv} state=absent force=yes'

削除後の論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

### 検証環境用設定ファイルの削除

検証環境の設定ファイルを削除します。

検証環境の設定ファイルを一覧表示します。

In [ ]:
target_dirs = ' '.join([f'{x}test-{update_project_tag}' for x in ['moodle', 'db']])
print(target_dirs)
!ansible {ugroup_name} -a 'chdir=/opt/moodle tree {target_dirs}'

検証環境の設定ファイルを削除します。

In [ ]:
!ansible {ugroup_name} -b -a 'chdir=/opt/moodle \
    rm -rf {target_dirs}'

削除されたことを確認します。

> 削除されていれば`[error opening dir]`と表示されます。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    tree {target_dirs}'

### 検証環境用コンテナイメージの削除

検証環境のためのコンテナイメージを削除します。

検証環境で利用していた削除対象となるコンテナイメージを表示します。

In [ ]:
target_image = f'local/{gvars["update_container_target"]}:{gvars["update_project_tag"]}-test'
!ansible {ugroup_name} -a 'docker images {target_image}'

検証環境のコンテナイメージを削除します。

In [ ]:
!ansible {ugroup_name} -a 'docker rmi {target_image}'

コンテナイメージが削除されたことを確認します。

> コンテナイメージが削除されていれば、ヘッダ以外は何も表示されません。

In [ ]:
!ansible {ugroup_name} -a 'docker images {target_image}'

## 新しい運用環境の起動

新しい運用環境のコンテナを起動します。

![新しい運用環境の起動](images/moodle-122-04.png)

### group_vars の更新

group_varsに記録している運用環境につけているタグの値を新しい環境の値に更新します。

In [ ]:
gvars = load_group_vars(ugroup_name)

# 更新対象のコンテナ種別
target_type = gvars['update_container_target']

# 新しい運用環境に関する情報
next_info = {
    'project_tag': gvars['update_project_tag'],
    f'{target_type}_image_name': f'local/{target_type}:{gvars["update_project_tag"]}',
}

# 現在の運用環境に関する情報
previous_info = {
    'project_tag': gvars['project_tag'],
    target_type + '_image_name': gvars[target_type + '_image_name'],
}

if 'previous_info_list' not in gvars:
    gvars['previous_info_list'] = []
gvars['previous_info_list'].insert(0, previous_info)

remove_group_vars(ugroup_name, 'update_project_tag', 'update_container_target')
update_group_vars(ugroup_name, previous_info_list=gvars['previous_info_list'], **next_info)
gvars = load_group_vars(ugroup_name)

### docker-compose.yml の更新

新しい運用環境のコンテナ構成を記述した `docker-compose.yml`をVCノードに配置します。

次のセルを実行するとローカル環境に新しい運用環境コンテナを追加した `docker-compose.yml` 
を作成し更新前との差分を表示します。また最後に表示されたリンクから更新後の `docker-compose.yml` 
を編集することもできます。

In [ ]:
%run scripts/edit_conf.py
update_docker_compose(ugroup_name)

更新した`docker-compose.yml`をVCノードに配置します。

In [ ]:
upload_docker_compose(ugroup_name)

### 新しい運用環境のコンテナ起動

現在のコンテナの実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

新しい運用環境のコンテナを起動します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose up -d'

新しい運用環境のコンテナを起動した後の、コンテナの実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

### リバースプロキシの設定

リバースプロキシの設定を変更して、新しい運用環境に切り替えます。

現在のリバースプロキシの設定ファイルを確認します。

In [ ]:
!ansible {ugroup_name} -a \
    'cat /opt/moodle/proxy/conf/moodle-proxy.conf'

次のセルを実行すると新しい運用環境を利用するように記述したApache HTTP Serverの設定ファイル
`moodle-proxy.conf`をローカル環境に作成します。作成後に現在の設定ファイルとの差分を表示します。
また、ローカル環境に生成した `moodle-proxy.conf` を編集するためのリンクを最後に表示します。

In [ ]:
%run scripts/edit_conf.py
update_proxy_conf(ugroup_name)

ローカル環境の変更後の `moodle-proxy.conf` をVCノードに配置します。
その後、設定ファイルの記述内容を反映させるためにリバースプロキシコンテナを再起動します。

In [ ]:
apply_proxy_conf(ugroup_name)

再起動後のコンテナの状態を確認します。`proxy`コンテナの状態が `Up` になっていることを確認してください。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

### メンテナンスモードの解除

In [ ]:
moodle_container = f'moodle-{gvars["project_tag"]}'
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose exec -T {moodle_container} \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## パッケージが更新されたことの確認

古い運用環境と新しい運用環境のパッケージの一覧を比較することで、更新されたパッケージを確認します。

古い運用環境のコンテナのパッケージ一覧を取得します。

In [ ]:
previous_project_tag = gvars['previous_info_list'][0]['project_tag']
previous_container = f'{target_type}-{previous_project_tag}'
package_old = !ansible {ugroup_name} -m shell -a 'chdir=/opt/moodle \
    docker exec -t {previous_container} \
    bash -c "( type rpm > /dev/null 2>&1 && rpm -qa ) || \
             ( type dpkg > /dev/null 2>&1 && dpkg -l ) " | sort'

新しい運用環境のコンテナのパッケージ一覧を取得します。

In [ ]:
current_container = f'{target_type}-{update_project_tag}'
package_new = !ansible {ugroup_name} -m shell -a 'chdir=/opt/moodle \
    docker exec -t {current_container} \
    bash -c "( type rpm > /dev/null 2>&1 && rpm -qa ) || \
             ( type dpkg > /dev/null 2>&1 && dpkg -l )" | sort'

パッケージ一覧の差分を確認します。

In [ ]:
from difflib import unified_diff

for line in unified_diff(package_old, package_new,
                         fromfile=previous_container, tofile=current_container):
    print(line)

## 古い運用環境のコンテナを停止する

古い運用環境のコンテナを停止します。

![古い運用環境の停止](images/moodle-122-05.png)

> ここではコンテナの停止のみ行い、古い運用環境の論理ボリュームや設定ファイルの削除は行いません。そのため、新しい運用環境で問題が生じた場合は「192-元の運用環境に戻す.ipynb」を実行することで元の運用環境に戻すことができます。また、新しい運用環境への移行が完了して古い運用環境が不要になった場合は「193-古い環境の削除.ipynb」を実行することで、古い運用環境の論理ボリュームなどのリソースを全て削除することができます。


古い運用環境のコンテナの停止を行います。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose up -d --remove-orphans'

実行しているコンテナの一覧を表示して、古い運用環境のコンテナが表示されないことを確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

## Moodle を利用できることを確認

新しい運用環境のMoodle環境にアクセスして正しく動作していることを確認します。

次のセルを実行するとMoodleのアドレスが表示されます。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/admin/index.php" target="_blank">{0}/admin/index.php</a>'.format(gvars['moodle_url']))